## 准备数据

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [9]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [10]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.uniform([784, 84], minval=-0.1, maxval=0.1), trainable=True)
        self.b1 = tf.Variable(tf.zeros([84]), trainable=True)
        self.W2 = tf.Variable(tf.random.uniform([84, 10], minval=-0.1, maxval=0.1), trainable=True)
        self.b2 = tf.Variable(tf.zeros([10]), trainable=True)
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 784))
        h1 = tf.nn.relu(x @ self.W1 + self.b1)
        h2 = tf.nn.relu(h1 @ self.W2 + self.b2)
        logits = h2

        return logits
        
model = myModel()

optimizer = optimizers.RMSprop(learning_rate=0.001)

## 计算 loss

In [11]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(grads, trainable_vars))
    # for g, v in zip(grads, trainable_vars):
    #     v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [12]:
def my_func(x):
    return np.sin(x)+np.exp(x)+np.tanh(x)

In [13]:
x = np.linspace(-10, 10, 1000)
y = my_func(x)

(x_train, y_train), (x_test, y_test) = (x[:784], y[:784]), (x[784:], y[784:])
train_data = (x_train, y_train)
test_data = (x_test, y_test)

for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

AttributeError: in user code:

    File "C:\Users\wonde\AppData\Local\Temp\ipykernel_22568\3758005975.py", line 16, in train_one_step  *
        loss = compute_loss(logits, y)
    File "C:\Users\wonde\AppData\Local\Temp\ipykernel_22568\3273897866.py", line 4, in compute_loss  *
        tf.nn.sparse_relu_cross_entropy_with_logits(

    AttributeError: module 'tensorflow._api.v2.nn' has no attribute 'sparse_relu_cross_entropy_with_logits'
